In [1]:
! pip3 install requests
! pip3 install beautifulsoup4
! python3 -m pip install bs4
! pip install geopy
! conda install -c conda-forge geocoder --yes 
! pip install folium==0.5.0

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np # library to handle data in a vectorized manner


import geocoder # import geocoder
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


import folium # plotting library

print('Libraries imported.')

     |████████████████████████████████| 115 kB 28.4 MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d1/41/e6495bd7d3781cee623ce23ea6ac73282a373088fcd0ddc809a047b18eae/beautifulsoup4-4.9.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/69/d82d04022f02733bf9a72bc3b96332d360c0c5307096d76f6bb7489f7e57/soupsieve-2.2.1-py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 112kB 24.0MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py

### Webscraping, Retreiving the URL and creating a Beautiful soup object

In [14]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(URL)
soup = BeautifulSoup(wiki.content, 'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table)) 
neighborhood = pd.DataFrame(df[0]) 
neighborhood.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


### Create a table with Postal code, Borough and Neighborhood information

In [15]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### The number of rows of the data frame

In [16]:
df.shape

(103, 3)

### Get the latitude and longitude coordinates for each postal code

In [9]:
latitude = []
longitude = []

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
for code in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    #print(code, g.latlng)

    while(g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
        #print(code, g.latlng)

    
    latitude.append(g.latlng[0])
    longitude.append(g.latlng[1])
    

In [ ]:
# Create data frame with latitude, longitude and merge to the original data frame 
location = {'Latitude':latitude, 'Longitude':longitude}
loc_data = pd.DataFrame(location)
loc_data.head()
df2 = pd.concat([df, loc_data], axis=1)
df2

In [25]:
# Access to Foursqure
CLIENT_ID = '5TZCGCPHMHGCQESK0F0P5CSP3VB0OMHIKVQPQQQLC5N0E3JF' # Foursquare ID
CLIENT_SECRET = 'GFNO5RAHFEQTYH2HCXMYGWRFWLG2H2BIKQIFQXOEJ3XVFUEG' #  Foursquare Secret
ACCESS_TOKEN = 'OPVEFAJVFAULR55SNHAOX1EPPAH4ZUHXMCC1OZPVRH4KJCOP' # FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5TZCGCPHMHGCQESK0F0P5CSP3VB0OMHIKVQPQQQLC5N0E3JF
CLIENT_SECRET:GFNO5RAHFEQTYH2HCXMYGWRFWLG2H2BIKQIFQXOEJ3XVFUEG


#### Assume you are staying at Sheraton Centre Toronto Hotel in Toronto.

#### Get latitude and longitude of Sheraton centre Toronto Hotel from the address.

In [26]:
# Location of Sheraton centre Toronto Hotel
address = '123 Queen St W, Toronto, ON'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and Longitude of Hotel:', latitude, longitude)

Latitude and Longitude of Hotel: 43.6511085 -79.38347444469197


#### Look for Japanese restaurants around the Hotel

In [28]:
search_query = 'Japanese'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4ae73054f964a5203ca921e3,Ki Modern Japanese + Bar,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1624470543,False,181 Bay St,at Wellington St. W,43.647223,-79.379374,"[{'label': 'display', 'lat': 43.647223, 'lng':...",544,M5J 2T3,CA,Toronto,ON,Canada,"[181 Bay St (at Wellington St. W), Toronto ON ...",NaN
1,4af63562f964a5203d0222e3,Bikkuri Japanese,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1624470543,False,36 King St. E,at Victoria,43.649564,-79.376328,"[{'label': 'display', 'lat': 43.649564308142, ...",600,NaN,CA,Toronto,ON,Canada,"[36 King St. E (at Victoria), Toronto ON]",NaN
2,4afa1f33f964a520e91622e3,Fune Japanese Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1624470543,False,100 Simcoe St.,at Pearl St.,43.648514,-79.386457,"[{'label': 'display', 'lat': 43.64851365544565...",375,NaN,CA,Toronto,ON,Canada,"[100 Simcoe St. (at Pearl St.), Toronto ON]",NaN
3,574ad72238fa943556d93b8e,Gyu-Kaku Japanese BBQ,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1624470543,False,81 Church St,at Adelaide St E,43.651422,-79.375047,"[{'label': 'display', 'lat': 43.65142227549791...",679,M5C 2G2,CA,Toronto,ON,Canada,"[81 Church St (at Adelaide St E), Toronto ON M...",NaN
4,4ae4b055f964a520229d21e3,Kyoto House Japanese Restaurant,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1624470543,False,143 Dundas St. West,NaN,43.655381,-79.385270,"[{'label': 'display', 'lat': 43.65538110598594...",497,NaN,CA,Toronto,ON,Canada,"[143 Dundas St. West, Toronto ON]",NaN


#### Clean the above table by keeping only columns that include venue name and location information

In [29]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Ki Modern Japanese + Bar,Japanese Restaurant,181 Bay St,at Wellington St. W,43.647223,-79.379374,"[{'label': 'display', 'lat': 43.647223, 'lng':...",544,M5J 2T3,CA,Toronto,ON,Canada,"[181 Bay St (at Wellington St. W), Toronto ON ...",NaN,4ae73054f964a5203ca921e3
1,Bikkuri Japanese,Sushi Restaurant,36 King St. E,at Victoria,43.649564,-79.376328,"[{'label': 'display', 'lat': 43.649564308142, ...",600,NaN,CA,Toronto,ON,Canada,"[36 King St. E (at Victoria), Toronto ON]",NaN,4af63562f964a5203d0222e3
2,Fune Japanese Restaurant,Japanese Restaurant,100 Simcoe St.,at Pearl St.,43.648514,-79.386457,"[{'label': 'display', 'lat': 43.64851365544565...",375,NaN,CA,Toronto,ON,Canada,"[100 Simcoe St. (at Pearl St.), Toronto ON]",NaN,4afa1f33f964a520e91622e3
3,Gyu-Kaku Japanese BBQ,Japanese Restaurant,81 Church St,at Adelaide St E,43.651422,-79.375047,"[{'label': 'display', 'lat': 43.65142227549791...",679,M5C 2G2,CA,Toronto,ON,Canada,"[81 Church St (at Adelaide St E), Toronto ON M...",NaN,574ad72238fa943556d93b8e
4,Kyoto House Japanese Restaurant,Sushi Restaurant,143 Dundas St. West,NaN,43.655381,-79.385270,"[{'label': 'display', 'lat': 43.65538110598594...",497,NaN,CA,Toronto,ON,Canada,"[143 Dundas St. West, Toronto ON]",NaN,4ae4b055f964a520229d21e3
5,Akashiro Japanese Restaurant & Bar,Sushi Restaurant,220 Yonge St.,in Toronto Eaton Centre Urban Eatery,43.655965,-79.380541,"[{'label': 'display', 'lat': 43.65596504293232...",590,NaN,CA,Toronto,ON,Canada,[220 Yonge St. (in Toronto Eaton Centre Urban ...,NaN,4fbbb3ace4b01e39077b4d55
6,Gyubee Japanese Grill,Japanese Restaurant,157 Dundas St W,NaN,43.655363,-79.384955,"[{'label': 'display', 'lat': 43.655363, 'lng':...",488,M5B 1E4,CA,Toronto,ON,Canada,"[157 Dundas St W, Toronto ON M5B 1E4]",NaN,5d782433f6e3190008b1a914
7,Omniwa Japanese Cuisine,Sushi Restaurant,214 King St. West,NaN,43.646109,-79.385934,"[{'label': 'display', 'lat': 43.64610862445569...",590,NaN,CA,Toronto,ON,Canada,"[214 King St. West, Toronto ON]",NaN,4c9ccede8afca093659ef015
8,Spring Rolls | Japanese Restaurant in Toronto,Theme Restaurant,"40 Dundas St W Box 36,",NaN,43.656105,-79.383495,"[{'label': 'display', 'lat': 43.65610515821675...",556,M5G 2C2,CA,Toronto,ON,Canada,"[40 Dundas St W Box 36,, Toronto ON M5G 2C2]",NaN,56dd9d68498eb4e5edcb30f9
9,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,43.649812,-79.379518,"[{'label': 'display', 'lat': 43.64981217424400...",349,M5H 1A1,CA,Toronto,ON,Canada,"[40 King St W (at Bay St), Toronto ON M5H 1A1]",Financial District,4beb2c0ba9900f475b7a1740


In [30]:
dataframe_filtered.name

0                          Ki Modern Japanese + Bar
1                                  Bikkuri Japanese
2                          Fune Japanese Restaurant
3                             Gyu-Kaku Japanese BBQ
4                   Kyoto House Japanese Restaurant
5                Akashiro Japanese Restaurant & Bar
6                             Gyubee Japanese Grill
7                           Omniwa Japanese Cuisine
8     Spring Rolls | Japanese Restaurant in Toronto
9                                       Ninki Sushi
10                                      Kenzo Ramen
11                Uncle Tetsu's Japanese Cheesecake
12                                             NAMI
Name: name, dtype: object

#### Show the location of the hotel and restaurants

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Sharaton centre Toronto Hotel

# add a red circle marker to represent the Sheraton Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Sheraton Centre Toronto Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

#### Find a restaurant has the highest rating. 

In [32]:
for i in dataframe_filtered['id']:
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
    except:
        print('This venue has not been rated yet.')

7.5
5.5
7.5
8.4
5.4
5.5
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
5.9
5.6
6.7
8.0


#### Clustering the neighbourhoods listed in DataFrame using Kmeans clustering 


In [147]:
# import libraries for KMeans clustering
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
%matplotlib inline

import random
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


#### Get latitude and longitude of Toronto

In [148]:

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
T_latitude = location.latitude
T_longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [149]:
# Visualizing all the Neighbourhoods of the above data frame named 'df2' 

map_toronto = folium.Map(location=[T_latitude, T_longitude],zoom_start=11)

for lat,lng,borough,neighbourhood in zip(df2['Latitude'],df2['Longitude'],df2['Borough'],df2['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

#### Clustering the neighbourhoods

In [131]:
#Setting up K-Means
clustering = df2.drop(['PostalCode','Borough','Neighborhood'],1)
k_means = KMeans(init = "k-means++", n_clusters = 5, n_init = 12)
k_means.fit(clustering)

#label each point
k_means_labels = k_means.labels_   
k_means_labels


array([2, 3, 0, 2, 0, 4, 3, 2, 0, 0, 2, 4, 3, 2, 0, 0, 0, 4, 3, 0, 0, 4,
       3, 0, 0, 0, 3, 2, 2, 0, 0, 0, 3, 2, 1, 0, 0, 0, 3, 2, 1, 0, 0, 0,
       3, 2, 1, 0, 0, 1, 1, 3, 2, 1, 0, 2, 1, 1, 3, 2, 1, 2, 2, 4, 1, 3,
       2, 2, 0, 4, 1, 3, 2, 2, 0, 0, 0, 1, 3, 0, 0, 4, 3, 0, 0, 3, 0, 0,
       4, 1, 3, 0, 0, 4, 1, 3, 0, 0, 4, 0, 0, 4, 4], dtype=int32)

In [140]:
#df2.drop('Cluster Labels', axis=1, inplace=True)


In [142]:
df2.insert(0, 'Cluster Labels', k_means_labels)
df2

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.75245,-79.32991
1,3,M4A,North York,Victoria Village,43.73057,-79.31306
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,0,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
...,...,...,...,...,...,...
98,4,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,0,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,0,M7Y,East Toronto Business,Enclave of M4L,43.64869,-79.38544
101,4,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [145]:
#get the coordinates of the cluster centers
#k_means_cluster_centers = k_means.cluster_centers_
#k_means_cluster_centers

array([[  2.        ,  43.667048  , -79.379816  ],
       [  0.        ,  43.72520786, -79.52373857],
       [  1.        ,  43.74859105, -79.39657684],
       [  3.        ,  43.76532   , -79.25183889],
       [  0.        ,  43.64646667, -79.50799333]])

#### Visualizing the clusters of neighbourhoods

In [146]:


map_clusters = folium.Map(location=[T_latitude, T_longitude], zoom_start=11)
kclusters = 5
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        df2['Latitude'], 
        df2['Longitude'], 
        df2['Neighborhood'], 
        df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters